In [ ]:
import numpy as np
import os 
import glob
import bdpy
import pickle
from torchvision import transforms
from PIL import Image
import torch
import torchvision.models as models
from torch.autograd import Variable

In [ ]:
fmri_data_table = [
    {'subject': 'sub-01',
     'data_file': '../data/fmri/sub-01_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'},
    {'subject': 'sub-02',
     'data_file': '../data/fmri/sub-02_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'},
    {'subject': 'sub-03',
     'data_file': '../data/fmri/sub-03_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'}
]

In [ ]:
image_dir = '../data/images/training'
image_file_pattern = '*.JPEG'

In [ ]:
fmri_data_bd = bdpy.BData(fmri_data_table[2]['data_file'])

In [ ]:
images_list = glob.glob(os.path.join(image_dir, image_file_pattern))  # List of image files (full path)
images_table = {os.path.splitext(os.path.basename(f))[0]: f
                    for f in images_list}
label_table = {os.path.splitext(os.path.basename(f))[0]: i + 1
                   for i, f in enumerate(images_list)}  

In [ ]:
fmri_labels = fmri_data_bd.get('Label')[:, 1].flatten()
fmri_labels = ['n%08d_%d' % (int(('%f' % a).split('.')[0]),
                                 int(('%f' % a).split('.')[1]))
                   for a in fmri_labels]

In [ ]:
fmri_data = fmri_data_bd.select(fmri_data_table[2]['roi_selector'])
print(np.max(fmri_data), np.min(fmri_data))
fmri_data_mean = np.mean(fmri_data, axis=0)
fmri_data_std = np.std(fmri_data, axis=0)
fmri_data = (fmri_data - fmri_data_mean) / fmri_data_std
print(np.max(fmri_data), np.min(fmri_data))


In [ ]:
import numpy as np

lower_bound = -13
upper_bound = 13 

array_shape = (6000, 9919)

random_array = np.random.normal(size=array_shape)

scaled_random_array = (random_array - random_array.min()) / (random_array.max() - random_array.min())
scaled_random_array = lower_bound + scaled_random_array * (upper_bound - lower_bound)
print(np.max(scaled_random_array), np.min(scaled_random_array))

In [ ]:
print(fmri_data.shape)

In [ ]:
np.save('subj3_fmri',fmri_data)

In [ ]:
np.save('subj3_baseline', scaled_random_array)

In [ ]:
fmri_data1 = fmri_data_bd.select('ROI_V1 = 1')
fmri_data2 = fmri_data_bd.select('ROI_V2 = 1')
fmri_data = np.concatenate((fmri_data1, fmri_data2), axis = 1)
print(fmri_data.shape)
fmri_data_mean = np.mean(fmri_data, axis=0)
fmri_data_std = np.std(fmri_data, axis=0)
fmri_data = (fmri_data - fmri_data_mean) / fmri_data_std

In [ ]:
fmri_data = fmri_data_bd.select('ROI_HVC = 1')
fmri_data_mean = np.mean(fmri_data, axis=0)
fmri_data_std = np.std(fmri_data, axis=0)
fmri_data = (fmri_data - fmri_data_mean) / fmri_data_std

In [ ]:
print(fmri_data.shape)

In [ ]:
np.save('subj3_fmri_HVC',fmri_data)

In [ ]:
fmri_data = fmri_data_bd.select('ROI_FFA = 1')
print(fmri_data.shape)
fmri_data_mean = np.mean(fmri_data, axis=0)
fmri_data_std = np.std(fmri_data, axis=0)
fmri_data = (fmri_data - fmri_data_mean) / fmri_data_std

In [ ]:
np.save('subj3_fmri_FFA',fmri_data)

In [ ]:
fmri_data = fmri_data_bd.select('ROI_PPA = 1')
print(fmri_data.shape)
fmri_data_mean = np.mean(fmri_data, axis=0)
fmri_data_std = np.std(fmri_data, axis=0)
fmri_data = (fmri_data - fmri_data_mean) / fmri_data_std

In [ ]:
np.save('subj3_fmri_PPA',fmri_data)

In [ ]:
filehandler = open('./image_fmri.pickle', 'wb')
pickle.dump(images_table, filehandler)

In [ ]:
print(images_table)

In [ ]:
image_features = np.empty((0, 9216))

alexnet = models.alexnet(pretrained=True)

alexnet_without_fc = torch.nn.Sequential(*(list(alexnet.features.children())))

alexnet_without_fc.eval()

preprocess = transforms.Compose([
    transforms.Resize(248),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
for i in range(len(fmri_labels)):
    print(i)
    image_path = images_table[fmri_labels[i]]
    image = Image.open(image_path)
    image = np.asarray(image)
    
    if image.ndim == 2:
        img_rgb = np.zeros((image.shape[0], image.shape[1], 3), dtype=image.dtype)
        img_rgb[:, :, 0] = image
        img_rgb[:, :, 1] = image
        img_rgb[:, :, 2] = image
        image = img_rgb
    input_tensor = preprocess(Image.fromarray(image))
    input_batch = input_tensor.unsqueeze(0)
    with torch.no_grad():
        features = alexnet_without_fc(input_batch)

    features = features.view(features.size(0), -1)
    image_features = np.vstack((image_features, features))


In [ ]:
np.save('./image_features', image_features)

In [ ]:
print(image_features[0][image_features[0]> 1])

In [ ]:
fmri_data_table_test = [
    {'subject': 'sub-01',
     'data_file': '../data/fmri/sub-01_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'},
    {'subject': 'sub-02',
     'data_file': '../data/fmri/sub-02_perceptionNaturalImageTraining_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'},
    {'subject': 'sub-03',
     'data_file': '../data/fmri/sub-03_perceptionNaturalImageTest_original_VC.h5',
     'roi_selector': 'ROI_VC = 1'}
]

In [ ]:
fmri_data_bd = bdpy.BData(fmri_data_table_test[2]['data_file'])

In [ ]:
image_dir_test = '../data/images/test'
image_file_pattern = '*.JPEG'

In [ ]:
images_list_test = glob.glob(os.path.join(image_dir, image_file_pattern))  # List of image files (full path)
images_table_test = {os.path.splitext(os.path.basename(f))[0]: f
                    for f in images_list}
label_table_test = {os.path.splitext(os.path.basename(f))[0]: i + 1
                   for i, f in enumerate(images_list)}  

In [ ]:
fmri_labels_test = fmri_data_bd.get('Label')[:, 1].flatten()
fmri_labels_test = ['n%08d_%d' % (int(('%f' % a).split('.')[0]),
                                 int(('%f' % a).split('.')[1]))
                   for a in fmri_labels_test]

In [ ]:
fmri_data_test = fmri_data_bd.select(fmri_data_table[2]['roi_selector'])

fmri_data_mean = np.mean(fmri_data, axis=0)
fmri_data_std = np.std(fmri_data, axis=0)
fmri_data_test = (fmri_data_test - fmri_data_mean) / fmri_data_std

In [ ]:
np.save('subj3_fmri_test',fmri_data_test)

In [ ]:
fmri_data1_test = fmri_data_bd.select('ROI_V1 = 1')
fmri_data2_test = fmri_data_bd.select('ROI_V2 = 1')
fmri_data_test = np.concatenate((fmri_data1_test, fmri_data2_test), axis = 1)
print(fmri_data_test.shape)
fmri_data_mean = np.mean(fmri_data_test, axis=0)
fmri_data_std = np.std(fmri_data_test, axis=0)
fmri_data_test = (fmri_data_test - fmri_data_mean) / fmri_data_std

In [ ]:
np.save('subj3_fmri_test_V1V2',fmri_data_test)

In [ ]:
fmri_data_test = fmri_data_bd.select('ROI_HVC = 1')
print(fmri_data_test.shape)
fmri_data_mean = np.mean(fmri_data_test, axis=0)
fmri_data_std = np.std(fmri_data_test, axis=0)
fmri_data_test = (fmri_data_test - fmri_data_mean) / fmri_data_std

In [ ]:
np.save('subj3_fmri_test_HVC',fmri_data_test)

In [ ]:
fmri_data_test = fmri_data_bd.select('ROI_FFA = 1')
print(fmri_data_test.shape)
fmri_data_mean = np.mean(fmri_data_test, axis=0)
fmri_data_std = np.std(fmri_data_test, axis=0)
fmri_data_test = (fmri_data_test - fmri_data_mean) / fmri_data_std

In [ ]:
np.save('subj3_fmri_test_FFA',fmri_data_test)

In [ ]:
fmri_data_test = fmri_data_bd.select('ROI_PPA = 1')
print(fmri_data_test.shape)
fmri_data_mean = np.mean(fmri_data_test, axis=0)
fmri_data_std = np.std(fmri_data_test, axis=0)
fmri_data_test = (fmri_data_test - fmri_data_mean) / fmri_data_std

In [ ]:
np.save('subj3_fmri_test_PPA',fmri_data_test)

In [ ]:
print(len(fmri_data_test))